# Avaliação de Medidas (In)Coerência do conjunto de dados FaQuAD usando BERT Pré-treinado

Este notebook, realiza testes de medidas de (in)coerência entre pares de documentos do conjunto de dados FaQuAD utilizando BERT Pré-treinado. 

A medida de um documento(D) é realizada pela média da medida dos pares de palavras. Todo o documento é submetido ao BERT e os embeddings da concatenação das 4 últimas camadas é recuperado. Cada documento é analisado pelas medidas **Ccos**, **Ceuc** e **Cman**. Estas medidas de documento utilizam as medidas de palavras adjancentes.Cada par de palavras do documento é analisado utilizando os embedding da palavra
(**wi**) e os embeddings da palavra(**wj**) no documento.

As seguintes medidas foram calculadas entre os embeddings das sentenças **wi** e **wj**:
- **Scos(wi,wj)** - Similaridade do cosseno entre a média dos embeddings Si e Sj.
- **Seuc(wi,wj)** - Distância euclidiana - usando a média dos embeddings Si e Sj das camadas especificadas
- **Sman(wi,wj)** - Distância de manhattan - usando a média dos embeddings wi e wj das camadas especificadas

As medidas são avalidas considerando a concatenação das 4 últimas do modelo BERT.

Características dos testes:
 - A avaliação das medidas são realizadas utilizando a implementação BERT da biblioteca Huggingface e os MCL BERT **Pré-treinado** no formato cased:
  - BERTimbau de tamanho Large 
  
 - Não é realizado ajuste fino do modelo.

Utiliza os arquivos para gerar as medidas:
- `original.zip`
- `originalpos.zip`
- `perturbado_pX_kY.zip`
- `perturbadopos_pX_kY.zip`
- `comparacao_palavra_pX_kY.zip`
- `comparacao_contexto_pX_kY.zip`

Nos nomes dos arquivos, `X` é o número de documentos perturbados e `Y` o valor de top `K` predições. 

----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers


**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

# 1 Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas
import time
import datetime

#marca o tempo de início do processamento.
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """    
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [ ]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))    

Classe(ModeloArgumentosMedida) de definição dos parâmetros do modelo para medida

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={'help': 'max seq len'},
    )    
    pretrained_model_name_or_path: str = field(
        default='neuralmind/bert-base-portuguese-cased',
        metadata={'help': 'nome do modelo pré-treinado do BERT.'},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={'help': 'define se o texto do modelo deve ser todo em minúsculo.'},
    )  
    output_attentions: bool = field(
        default=False,
        metadata={'help': 'habilita se o modelo retorna os pesos de atenção.'},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={'help': 'habilita gerar as camadas ocultas do modelo.'},
    )
    use_wandb : bool = field(
        default=True,
        metadata={'help': 'habilita o uso do wandb.'},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={'help': 'habilita o salvamento do resultado da avaliação.'},
    )     
    salvar_medicao : bool = field(
        default=False,
        metadata={'help': 'habilita o salvamento da medicao.'},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={'help': 'habilita o carragamento de mcl ajustado.'},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    estrategia_medida: int = field(
        default=0, # 0 - MEAN estratégia média / 1 - MAX  estratégia maior
        metadata={'help': 'Estratégia de cálculo da médida dos embeddings.'},
    )
    filtro_palavra: int = field(
        default=0, # 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
        metadata={'help': 'Define o filtro de palavras das sentenças para gerar os embeddings.'},
    )

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4  Identificando o ambiente Colab

In [ ]:
# Se estiver executando no Google Colaboratory.
import sys

# Retorna true ou false se estiver no Google Colaboratory.
IN_COLAB = 'google.colab' in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

2022-07-12 12:06:55,641 : INFO : NumExpr defaulting to 2 threads.
2022-07-12 12:06:56,175 : INFO : Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.
    
      Retorno:
        `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

2022-07-12 12:07:00,416 : INFO : Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

2022-07-12 12:07:00,460 : INFO : Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

2022-07-12 12:07:00,466 : INFO : Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
2022-07-12 12:07:00,469 : INFO : e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Mounted at /content/drive


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 146 kB 53.0 MB/s 
     |████████████████████████████████| 181 kB 56.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=9ba84c1455e368e9cd47a1a390aa0376ef21d57525b746456afcd5d945b4564b
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 48.5 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 44.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.1
    Uninstalling spacy-3.3.1:
      Successfully uninstalled spacy-3.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.3.0 requires spacy<3.4.0,>=3.3.0.dev0, but you have spacy 3.2.0 which is incompatible.


## 1.9 Instalação do BERT

Instala a interface pytorch para o BERT by Hugging Face. 

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

In [ ]:
!pip install -U transformers==4.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 53.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.8 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=23faef925cd2c330a303c7ec0e193ac96c4a465f2f7a385955a7da38b132007b
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


# 2 Parametrização

## Gerais

In [ ]:
# Nome base das saidas do projeto
NOME_BASE_SAIDA = "MedidaCoerenciaFaquad_v1"

# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 1

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 1

# Equação de coerência (0 - Palavras adjacentes / 1 - Combinação de todas as palavras / 2 - Palavra e contexto) 
EQUACAO_MEDIDA_STR = ["ADJACENTES","COMBINAÇÃO","CONTEXTO"]
EQUACAO_MEDIDA = [0, 1, 2]

# Estratégias a serem avaliadas (0 - Mean / 1 - Max) para as palavras formadas por mais de um token do BERT
ESTRATEGIA_MEDIDA_STR = ["MEAN", "MAX"]
ESTRATEGIA_MEDIDA = [0, 1]

# Filtro de palavras das sentenças[0,1,2] 'TAP,SSW,SVS'
FILTRO_PALAVRAS_STR = ["TODAS_AS_PALAVRAS", 
                       "SEM_STOPWORDS",                        
                       "SOMENTE_VERBOS_SUBSTANTIVOS"]
FILTRO_PALAVRAS = [0, 1, 2]

## Específicos

Parâmetros do modelo

In [ ]:


# Definição dos parâmetros do Modelo
model_args = ModeloArgumentosMedida(     
    max_seq_len = 512,    
    pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",    
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",
    versao_modelo_spacy = "3.2.0",
    do_lower_case = False, # default True
    output_attentions = False, # default False
    output_hidden_states = True, # default False
    use_wandb = True,    
    salvar_medicao = True, #Salva o resultado da medição
    salvar_avaliacao = True, # Salva o resultado da avaliação das medições
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path. 
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.    
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara. 
    estrategia_medida = 0, # Atributo usado para os logs do wandb. 0 - MEAN estratégia média / 1 - MAX  estratégia maior
    filtro_palavra = 0, # Atributo usado para os logs do wandb. 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "FAQUAD"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

## Inicialização diretórios

Diretório base local

In [ ]:
# Importando as bibliotecas.
import os

def criaDiretorioLocal():

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):  
      # Cria o diretório
      os.makedirs(dirbase)    
      logging.info("Diretório criado: {}.".format(dirbase))
  else:    
      logging.info("Diretório já existe: {}.".format(dirbase))

In [ ]:
criaDiretorioLocal()

2022-07-12 12:08:13,861 : INFO : Diretório criado: /content/FAQUAD.


Diretório para conter as os resultados das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedidacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "/validacao_medicao_palavra"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedidacao()

2022-07-12 12:08:14,359 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/FAQUAD//validacao_medicao_palavra.


Diretório para conter os arquivos da avaliação

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioAvaliacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Avaliacao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioAvaliacao()

2022-07-12 12:08:14,602 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/FAQUAD/validacao_medicao_palavra/Avaliacao.


Diretório para conter os arquivos das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedicao():

  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Medicao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedicao()

2022-07-12 12:08:14,647 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/FAQUAD/validacao_medicao_palavra/Medicao.


# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivoTar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivoTar.extractall(DIRETORIO_COHEBERT)    
    arquivoTar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

2022-07-12 12:08:15,965 : INFO : Download do arquivo do modelo do spaCy.
2022-07-12 12:08:16,329 : INFO : Download do arquivo: FAQUAD/pt_core_news_lg-3.2.0.tar.gz.


  0%|          | 0.00/577M [00:00<?, ?B/s]

2022-07-12 12:09:03,161 : INFO : Descompactando o arquivo do modelo do spaCy.
2022-07-12 12:09:14,897 : INFO : spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [ ]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).
    
      Parâmetros:
        `nlp` - Um modelo spaCy carregado.           
    """
    
    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords 

Lista dos stopwords

In [ ]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

2022-07-12 12:09:14,930 : INFO : Quantidade de stopwords: 416.


{'meses', 'faço', 'sétimo', 'naquele', 'vos', 'parece', 'tentaram', 'seis', 'mas', 'quarta', 'somos', 'custa', 'momento', 'dar', 'é', 'quieta', 'fazeis', 'estivemos', 'podem', 'nós', 'geral', 'os', 'daquele', 'quanto', 'pontos', 'nuns', 'sim', 'tive', 'também', 'lado', 'apoio', 'através', 'nada', 'nova', 'cedo', 'eles', 'vem', 'vocês', 'estão', 'antes', 'tudo', 'dois', 'lá', 'seus', 'fui', 'terceira', 'temos', 'quero', 'somente', 'podia', 'teu', 'em', 'puderam', 'tiveste', 'primeiro', 'oitava', 'tenho', 'minhas', 'bastante', 'aos', 'deverá', 'nossa', 'tiveram', 'agora', 'longe', 'enquanto', 'grande', 'lugar', 'usa', 'sem', 'apontar', 'grandes', 'algumas', 'num', 'vai', 'vais', 'pouca', 'das', 'era', 'porquê', 'ter', 'meio', 'dão', 'vossas', 'dá', 'ademais', 'esse', 'quarto', 'sexto', 'assim', 'por', 'aquela', 'for', 'menos', 'falta', 'valor', 'fazes', 'sempre', 'portanto', 'ser', 'contra', 'cada', 'sou', 'fomos', 'quer', 'a', 'pela', 'estar', 'quinto', 'na', 'povo', 'aqueles', 'estado'

### getVerbos
Localiza os verbos da sentença

In [ ]:
# Import das bibliotecas.
import spacy   
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                                  
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar   
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ] 

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [               
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                   
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal       
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ] 

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar 
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}}, 
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar 
               ] 

matcherv = Matcher(nlp.vocab)
         
matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):    
  #Processa o período
  doc1 = nlp(periodo.text)
  
  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))
  
  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()
  
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [ ]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades    
  novodic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}
    
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [ ]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):
    
  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))
 
  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [ ]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [ ]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [ ]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  listatokens = []
  listapos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:    
    listatokens.append(token.text)
    listapos.append(token.pos_)
    
  return listatokens, listapos

### Tradução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [ ]:
#dicionário que contêm pos tag universal e suas explicações
palavrauniversal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}
  
#Explica a POS
def traduzPOSPalavraUniversal(palavra):
  if palavra in palavrauniversal_dict.keys():
      traduzido = palavrauniversal_dict[palavra]
  else:
      traduzido = "NA" 
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [ ]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [ ]:
def getSentencaSalientePOS(sentenca, pos, classe_saliente=["NOUN"]):
  
  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especificado
    if pos[i] in classe_saliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [ ]:
def removeStopWord(documento, stopwords):
  
  # Remoção das stopwords do documento
  documentoSemStopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documento_limpo = " ".join(documentoSemStopwords)

  # Retorna o documento
  return documento_limpo

# 4 Funções auxiliares

## getNomeModeloBERT

In [ ]:
def getNomeModeloBERT(model_args):
    '''    
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"
    
    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"        
    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"
        else:
            if 'bert' in model_args.pretrained_model_name_or_path:
                MODELO_BERT = "_BERT"  
            
    return MODELO_BERT

## getTamanhoBERT

In [ ]:
def getTamanhoBERT(model_args):
    '''    
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''
    
    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"
    
    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"
        
    return TAMANHO_BERT  

## concatenaListas

In [ ]:
def concatenaListas(lista, pos=1):
  lista_concat = []

  for x in lista:
      lista_concat = lista_concat + x[pos]
  
  return lista_concat

## encontrarIndiceSubLista 

Retorna os índices de início e fim da sublista na lista

In [ ]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):

    """
      Localiza os índices de início e fim de uma sublista em uma lista.
    
      Parâmetros:
      `lista` - Uma lista.
      `sublista` - Uma sublista a ser localizada na lista.
    """    
    # https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

    # Recupera o tamanho da lista 
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)    
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
        for j in range(n):
            if lista[i - j] != sublista[-j - 1]:
                i += skip.get(lista[i], n)
                break
        else:
            indice_inicio = i - n + 1
            indice_fim = indice_inicio + len(sublista)-1
            return indice_inicio, indice_fim
    return -1, -1

# 5 Comparar documentos





## 5.1 Carregamento dos arquivos de dados originais

### 5.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "originalpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

NOME_ARQUIVO_PERTURBADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_COMPACTADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"
NOME_ARQUIVO_PERTURBADO_POS = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.1.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

2022-07-12 12:09:15,307 : INFO : Diretório já existe: /content/FAQUAD.


### 5.1.3 Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" "$DIRETORIO_LOCAL"
 
  logging.info("Terminei a cópia.")

2022-07-12 12:09:17,485 : INFO : Terminei a cópia.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

2022-07-12 12:09:18,360 : INFO : Terminei a descompactação.


### 5.1.4 Carregamento das lista com os dados dos arquivos originais

#### Carrega o arquivo dos dados originais e POS

In [ ]:
# Importando as bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-07-12 12:09:18,399 : INFO : TERMINADO ORIGINAIS: 900.
2022-07-12 12:09:18,400 : INFO : TERMINADO ORIGINAIS POS: 900.


In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento
353,3dc66f2440a446f3b6e118cf4783f71a,['Qual foi a necessidade que fez a lei da cria...,Qual foi a necessidade que fez a lei da criaçã...
536,8c315266ef62458c9b670c25963cea9b,['Em que ano foi reconhecido o mestrado profis...,Em que ano foi reconhecido o mestrado profissi...
641,3dde822521e843fb894d273aa07c8deb,['O turno matutino compreende as atividades re...,O turno matutino compreende as atividades real...
602,b82ae73f31f4463690ee784902d45094,['Onde devem ser lançadas as notas e as frequê...,Onde devem ser lançadas as notas e as frequênc...
799,7c60d7ee13eb4e2baa1cb28a3feaa398,['Qual a principal fonte de renda das universi...,Qual a principal fonte de renda das universida...


In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
177,bb9ce6fcdda2466ca777711c04188028,"[[['Até', 'quando', 'o', 'ENIAC', 'foi', 'mant..."
152,b74772acc3f149498a35481c365dbf97,"[[['Quais', 'as', 'formas', 'mais', 'comuns', ..."
537,004ecef2d5ed419785ede3b016138f10,"[[['Quando', 'o', 'doutorado', 'passou', 'a', ..."
32,d55d83a33e9343ddad53d78a2d2dac9e,"[[['Com', 'o', 'que', 'as', 'atividades', 'com..."
700,c62511babcc444ae9f76d9c672ab30a7,"[[['O', 'que', 'deve', 'ser', 'anexado', 'ao',..."


#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-07-12 12:09:18,543 : INFO : TERMINADO CORREÇÃO ORIGINAIS: 900.
2022-07-12 12:09:18,545 : INFO : TERMINADO CORREÇÃO ORIGINAIS POS: 900.


#### Criando dados indexados originais

In [ ]:
# Faz uma cópia da lista não indexada
lista_documentos_originais_indexada = lista_documentos_originais.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_documentos_originais_indexada.set_index(["id"], inplace=True)
lista_documentos_originais_indexada.head()

,sentencas,documento
id,,
4918ff59e83343b88c1fd3a204f55a3b,[Como pode ser definido um sistema de informaç...,Como pode ser definido um sistema de informação?
fb885a2f0a624520a6586476563715e8,[O que as soluções tecnológicas automatizam?],O que as soluções tecnológicas automatizam?
a8d728463e2f4fe792c6c756e0fd14a4,"[Por que a preocupação com a coleta, armazenam...","Por que a preocupação com a coleta, armazename..."
927902205b9f494881848f7fc065c347,[Qual um componente indispensável nas organiza...,Qual um componente indispensável nas organizaç...
ee17827309224fa8acfb23a05e2fa853,[Em que se concentram as responsabilidades do ...,Em que se concentram as responsabilidades do a...


In [ ]:
# Faz uma cópia da lista não indexada
lista_documentos_originais_pos_indexada = lista_documentos_originais_pos.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_documentos_originais_pos_indexada.set_index(["id"], inplace=True)
lista_documentos_originais_pos_indexada.head()

,pos_documento
id,
4918ff59e83343b88c1fd3a204f55a3b,"[[[Como, pode, ser, definido, um, sistema, de,..."
fb885a2f0a624520a6586476563715e8,"[[[O, que, as, soluções, tecnológicas, automat..."
a8d728463e2f4fe792c6c756e0fd14a4,"[[[Por, que, a, preocupação, com, a, coleta, ,..."
927902205b9f494881848f7fc065c347,"[[[Qual, um, componente, indispensável, nas, o..."
ee17827309224fa8acfb23a05e2fa853,"[[[Em, que, se, concentram, as, responsabilida..."


#### Carrega o arquivo dos dados perturbados e POS

In [ ]:
# Abre o arquivo e retorna o DataFrame
lista_documentos_perturbados = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO, sep=";", encoding="UTF-8")
lista_documentos_perturbados_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO PERTURBADOS: {}.".format(len(lista_documentos_perturbados)))
logging.info("TERMINADO PERTURBADOS POS: {}.".format(len(lista_documentos_perturbados_pos)))

2022-07-12 12:09:19,671 : INFO : TERMINADO PERTURBADOS: 90000.
2022-07-12 12:09:19,673 : INFO : TERMINADO PERTURBADOS POS: 90000.


In [ ]:
lista_documentos_perturbados.sample(5)

,id,perturbado,documentoPerturbado,sentencas
45556,cefca7c5620041f2a475bd7f9060f591_pert_56,['Por quem é apaixonada a pesquisa acadêmica ?'],Por quem é apaixonada a pesquisa acadêmica ?,"[['Por quem é [MASK] a pesquisa acadêmica ?', ..."
16198,d36826bb9cbb40b7a47074cff2c2c014_pert_98,['Qual linguagem de programação é parecida com...,Qual linguagem de programação é parecida com a...,[['Qual linguagem de programação é parecida co...
39232,1a80267bbdef4f059d05e1cb26e1ce61_pert_32,['De quando é a nota nº 342 ?'],De quando é a nota nº 342 ?,"[['De quando é a [MASK] nº 342 ?', 'resolução'..."
57606,1567f487837a46728d0830c6fb8ff3a5_pert_6,['Como são selecionados os professores de ensi...,Como são selecionados os professores de ensino...,[['Como são [MASK] os professores de ensino fu...
31375,2f8b4896dd3d4692a4e286fb40826df5_pert_75,['O que a gestante deve descrever para requisi...,O que a gestante deve descrever para requisita...,[['O que a gestante deve [MASK] para requisita...


In [ ]:
lista_documentos_perturbados_pos.sample(5)

,id,pos_documento
27898,24d99baaba914b819bba40afc4699db1_pert_98,"[[['Em', 'que', 'ano', 'foi', 'chamado', 'o', ..."
56451,bd1eed09be974557ad39a40f126818eb_pert_51,"[[['O', 'que', 'é', 'necessário', 'para', 'que..."
57025,a35303d6844a4a58adc576f87c7ecb04_pert_25,"[[['Qual', 'a', 'maior', 'concentração', 'sind..."
15042,5b5f263d3c1546ddaee84a1a848d2704_pert_42,"[[['O', 'que', 'é', 'esse', '?'], ['PRON', 'PR..."
52648,7a50b2ccd7b9424f87bf312e60fc9a44_pert_48,"[[['Qual', 'modelo', 'computacional', 'propôs'..."


#### Corrigir os tipos de colunas dos dados perturbados e POS

Em dados perturbados:
- coluna 1 - `perturbado` carregadas do arquivo vem como string e não como lista.
- coluna 3 - `sentencas` carregadas do arquivo vem como string e não como lista.

Em dados perturbados pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_perturbados["perturbado"] = lista_documentos_perturbados["perturbado"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)
lista_documentos_perturbados["sentencas"] = lista_documentos_perturbados["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_perturbados_pos["pos_documento"] = lista_documentos_perturbados_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO PERTURBADO: {}.".format(len(lista_documentos_perturbados)))
logging.info("TERMINADO CORREÇÃO PERTURBADO POS: {}.".format(len(lista_documentos_perturbados_pos)))

2022-07-12 12:09:29,588 : INFO : TERMINADO CORREÇÃO PERTURBADO: 90000.
2022-07-12 12:09:29,591 : INFO : TERMINADO CORREÇÃO PERTURBADO POS: 90000.


#### Criando dados indexados perturbados

In [ ]:
# Faz uma cópia da lista não indexada
lista_documentos_perturbados_indexada = lista_documentos_perturbados.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_documentos_perturbados_indexada.set_index(["id"], inplace=True)
lista_documentos_perturbados_indexada.head()

,perturbado,documentoPerturbado,sentencas
id,,,
4918ff59e83343b88c1fd3a204f55a3b_pert_0,[Como pode ser definido um sistema de informaç...,Como pode ser definido um sistema de informação ?,[[Como [MASK] ser definido um sistema de infor...
4918ff59e83343b88c1fd3a204f55a3b_pert_1,[Como deve ser definido um sistema de informaç...,Como deve ser definido um sistema de informação ?,[[Como [MASK] ser definido um sistema de infor...
4918ff59e83343b88c1fd3a204f55a3b_pert_2,[Como poderia ser definido um sistema de infor...,Como poderia ser definido um sistema de inform...,[[Como [MASK] ser definido um sistema de infor...
4918ff59e83343b88c1fd3a204f55a3b_pert_3,[Como deveria ser definido um sistema de infor...,Como deveria ser definido um sistema de inform...,[[Como [MASK] ser definido um sistema de infor...
4918ff59e83343b88c1fd3a204f55a3b_pert_4,[Como poderá ser definido um sistema de inform...,Como poderá ser definido um sistema de informa...,[[Como [MASK] ser definido um sistema de infor...


In [ ]:
# Faz uma cópia da lista não indexada
lista_documentos_perturbados_pos_indexada = lista_documentos_perturbados_pos.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_documentos_perturbados_pos_indexada.set_index(["id"], inplace=True)
lista_documentos_perturbados_pos_indexada.head()

,pos_documento
id,
4918ff59e83343b88c1fd3a204f55a3b_pert_0,"[[[Como, pode, ser, definido, um, sistema, de,..."
4918ff59e83343b88c1fd3a204f55a3b_pert_1,"[[[Como, deve, ser, definido, um, sistema, de,..."
4918ff59e83343b88c1fd3a204f55a3b_pert_2,"[[[Como, poderia, ser, definido, um, sistema, ..."
4918ff59e83343b88c1fd3a204f55a3b_pert_3,"[[[Como, deveria, ser, definido, um, sistema, ..."
4918ff59e83343b88c1fd3a204f55a3b_pert_4,"[[[Como, poderá, ser, definido, um, sistema, d..."


### 5.1.5 Gerando pares de documentos originais e perturbados / Documento Original(1) e Documento Perturbado(0)


In [ ]:
# Import das bibliotecas.
import ast
from tqdm.notebook import tqdm as tqdm_notebook

print("Processando",len(lista_documentos_originais),"documentos originais")

lista_documentos_agrupados = []

# Barra de progresso dos documentos
lista_documentos_originais_bar = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_originais))

# Percorre os documentos
for i, linha_documento in lista_documentos_originais_bar: 
  #if i < 2:
    #print("linha_documento:",linha_documento)
    # Recupera o id do documento
    id_documento_original = linha_documento[0]     
    #print("id_documento_original:",id_documento_original)     
 
    # Carrega a lista das sentenças do documento
    lista_sentencas_original = linha_documento[1]    
    #print("\lista_sentencas_original:",lista_sentencas_original)
    #print("len(lista_sentencas_original):",len(lista_sentencas_original)) 

    # Carrega o documento original
    documento_original = linha_documento[2]    
    #print("\documento_original:",documento_original)
    
    # Recupera a POS do documento original
    tokens_original = []
    tokens_original_pos = []    
    reg_original_pos = lista_documentos_originais_pos_indexada.loc[id_documento_original] 
    # print("reg_original_pos:",reg_original_pos)    
    pos_documento_original = reg_original_pos['pos_documento']
    for i, linha2 in enumerate(pos_documento_original):
      
      tokens_original.append(linha2[0])
      tokens_original_pos.append(linha2[1])

    # Percorre os documentos perturbados apartir do original
    for j in range(0, model_args.documentos_perturbados):
  
        # Id do documento perturbado
        id_perturbado = str(id_documento_original) + "_pert_" + str(j)
        #print("id_perturbado:", id_perturbado)

        # Recupera o documento perturbado apartir do id original
        reg_documento_perturbado = lista_documentos_perturbados_indexada.loc[id_perturbado]
        # Recupera a sentença do documento perturbado
        lista_sentencas_perturbado = reg_documento_perturbado["perturbado"]
        #print("\lista_sentencas_perturbado:",lista_sentencas_perturbado)
        #print("len(lista_sentencas_perturbado):",len(lista_sentencas_perturbado)) 

        # Carrega o documento perturbado
        documento_perturbado = reg_documento_perturbado[2]    
        #print("\documento_perturbado:",documento_perturbado)

        # Recupera a POS do documento perturbado
        tokens_perturbado = []
        tokens_perturbado_pos = []        
        reg_perturbado_pos = lista_documentos_perturbados_pos_indexada.loc[id_perturbado]
        #print("reg_perturbado_pos:",reg_perturbado_pos)
        pos_documento_perturbado = reg_perturbado_pos['pos_documento']
        for i, linha2 in enumerate(pos_documento_perturbado):
          tokens_perturbado.append(linha2[0])
          tokens_perturbado_pos.append(linha2[1])

        # Guarda o agrupamento de original e perturbado                
        lista_documentos_agrupados.append([id_documento_original, 
                                           lista_sentencas_original, 
                                           documento_original, 
                                           tokens_original, 
                                           tokens_original_pos, 
                                           id_perturbado, 
                                           lista_sentencas_perturbado, 
                                           documento_perturbado, 
                                           tokens_perturbado, 
                                           tokens_perturbado_pos])

logging.info("TERMINADO AGRUPAMENTO: {}.".format(len(lista_documentos_agrupados)))

Processando 900 documentos originais


Documentos:   0%|          | 0/900 [00:00<?, ? documento/s]

2022-07-12 12:09:46,577 : INFO : TERMINADO AGRUPAMENTO: 90000.


#### Converte a lista em um dataframe

Atributos do dataframe:
Atributos do dataframe:
0. 'id_original' - Nome do arquivo original
1. 'sentencas_originais' - Lista das sentenças do documento original
2. 'documento_original' - Documento original
3. 'tokens_original' - Tokens do documento original
4. 'pos_original' - Postagging do documento original
5. 'id_perturbado' - Nome do arquivo perturbado
6. 'sentencas_perturbadas' - Lista das sentenças do documento perturbado
7. 'documento_perturbado' - Documento perturbado
8. 'tokens_perturbado' - Tokens do documento perturbado
9. 'pos_perturbado' - Postagging do documento perturbado

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Converte a lista em um dataframe.
df_lista_documentos_agrupados = pd.DataFrame.from_records(lista_documentos_agrupados, 
                                                         columns=['id_original',
                                                                  'sentencas_original',
                                                                  'documento_original',
                                                                  'tokens_original',
                                                                  'pos_original',
                                                                  'id_perturbado',
                                                                  'sentencas_perturbado',
                                                                  'documento_perturbado',
                                                                  'tokens_perturbado', 
                                                                  'pos_perturbado'])

# Número de linhas carregadas do arquivo.
logging.info('Total de registros              : {}'.format(len(df_lista_documentos_agrupados)))

2022-07-12 12:09:46,650 : INFO : Total de registros              : 90000


In [ ]:
df_lista_documentos_agrupados.sample(5)

,id_original,sentencas_original,documento_original,tokens_original,pos_original,id_perturbado,sentencas_perturbado,documento_perturbado,tokens_perturbado,pos_perturbado
31006,879f1d34d55b42fb9a0250288ca2537e,[Qual o objetivo do estágio?],Qual o objetivo do estágio?,"[[Qual, o, objetivo, do, estágio, ?]]","[[DET, DET, NOUN, ADP, NOUN, PUNCT]]",879f1d34d55b42fb9a0250288ca2537e_pert_6,[Qual o objetivo do site ?],"[[Qual o objetivo do [MASK] ?, estágio, site, ...","[[Qual, o, objetivo, do, site, ?]]","[[DET, DET, NOUN, ADP, NOUN, PUNCT]]"
85987,c31fd29b500e4fedbbd80604f4b91d05,[quais são as fases da renovação de matrícula?],quais são as fases da renovação de matrícula?,"[[quais, são, as, fases, da, renovação, de, ma...","[[PRON, AUX, DET, NOUN, ADP, NOUN, ADP, NOUN, ...",c31fd29b500e4fedbbd80604f4b91d05_pert_87,[quais são as fases da entrega de matrícula ?],"[[quais são as fases da [MASK] de matrícula ?,...","[[quais, são, as, fases, da, entrega, de, matr...","[[PRON, AUX, DET, NOUN, ADP, NOUN, ADP, NOUN, ..."
17753,bb9ce6fcdda2466ca777711c04188028,[Até quando o ENIAC foi mantido em segredo?],Até quando o ENIAC foi mantido em segredo?,"[[Até, quando, o, ENIAC, foi, mantido, em, seg...","[[ADV, ADV, DET, PROPN, AUX, VERB, ADP, NOUN, ...",bb9ce6fcdda2466ca777711c04188028_pert_53,[Até quando o ENIAC foi seguido em segredo ?],"[[Até quando o ENIAC foi [MASK] em segredo ?, ...","[[Até, quando, o, ENIAC, foi, seguido, em, seg...","[[ADV, ADV, DET, PROPN, AUX, VERB, ADP, NOUN, ..."
46939,8e0ef2d36bbd4780b1aa4c8fe244099a,[Em qual ambiente a pesquisa empírica é feita?],Em qual ambiente a pesquisa empírica é feita?,"[[Em, qual, ambiente, a, pesquisa, empírica, é...","[[ADP, PRON, ADJ, DET, NOUN, ADJ, AUX, VERB, P...",8e0ef2d36bbd4780b1aa4c8fe244099a_pert_39,[Em qual ambiente a pesquisa empírica é atraen...,[[Em qual ambiente a pesquisa empírica é [MASK...,"[[Em, qual, ambiente, a, pesquisa, empírica, é...","[[ADP, PRON, ADJ, DET, NOUN, ADJ, AUX, ADJ, PU..."
15631,34d2f2b4719b4aa793e5118e9fe80832,[O que o compartilhamento de tempo da CPU gera?],O que o compartilhamento de tempo da CPU gera?,"[[O, que, o, compartilhamento, de, tempo, da, ...","[[PRON, PRON, DET, NOUN, ADP, NOUN, ADP, PROPN...",34d2f2b4719b4aa793e5118e9fe80832_pert_31,[O que o compartilhamento de tempo da CPU expl...,[[O que o compartilhamento de tempo da CPU [MA...,"[[O, que, o, compartilhamento, de, tempo, da, ...","[[PRON, PRON, DET, NOUN, ADP, NOUN, ADP, PROPN..."


Apaga as listas que não serão mais utilizadas

In [ ]:
del lista_documentos_originais
del lista_documentos_originais_pos
del lista_documentos_perturbados
del lista_documentos_perturbados_pos

## 5.2 Carregamento do arquivo de dado comparação entre palavras

### 5.2.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_COMPARACAO_PALAVRA = "comparacao_palavra_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO = "comparacao_palavra_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.2.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}".format(dirbase))
else:    
    logging.info("Diretório já existe: {}".format(dirbase))

2022-07-12 12:09:46,741 : INFO : Diretório já existe: /content/FAQUAD


### 5.2.3 Copia e descompacta os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  # Copia o arquivo de comparações do google drive para a diretório local
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO"
  
  logging.info("Terminei a cópia!")

2022-07-12 12:09:49,120 : INFO : Terminei a cópia!


Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Descompactação realizada!")

2022-07-12 12:09:57,378 : INFO : Descompactação realizada!


### 5.2.4 Carregamento das lista com os dados do arquivo


In [ ]:
#Biblioteca
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_comparacao_palavra = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_COMPARACAO_PALAVRA, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_comparacao_palavra)))

2022-07-12 12:10:10,498 : INFO : TERMINADO ORIGINAIS: 5598531.


In [ ]:
lista_comparacao_palavra.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,index_wj,wj,pos_j,cos_mean,euc_mean,man_mean,cos_max,euc_max,man_max
3309164,96671a8be4ff432598bd8a3e60ddf6e8_pert_84,54120,0,10,mestrado,NOUN,11,sem,SCONJ,0.538079,39.901379,1849.3472,0.538079,39.901379,1849.3472
552218,ee2a6e83216b47f5b390be193221f4e5_pert_87,9279,0,4,moderno,ADJ,5,começou,VERB,0.688529,33.217636,1608.8656,0.688529,33.217636,1608.8656
4119000,4d837be563594345b5ccb94612982dd0_pert_5,66969,0,0,Onde,NOUN,5,aproveitamento,NOUN,0.627816,35.392292,1681.7849,0.627816,35.392292,1681.7849
3704260,38643f4a251d492e8558039f49c54a8f_pert_72,60067,0,11,outro,DET,14,graduação,NOUN,0.636436,32.649529,1554.0223,0.636436,32.649529,1554.0223
120446,88930242282f4a4c9d65c139f2bf81d8_pert_95,1813,0,6,de,ADP,12,de,ADP,0.636861,32.689137,1618.8673,0.636861,32.689137,1618.8673


### 5.2.5 Criando dados indexados

In [ ]:
# Faz uma cópia da lista não indexada
lista_comparacao_palavra_indexado = lista_comparacao_palavra.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_comparacao_palavra_indexado.set_index(["id", "index_sentenca", "index_wi", "index_wj"], inplace=True)
lista_comparacao_palavra_indexado.head()

index_documento  \
id                               index_sentenca index_wi index_wj                    
4918ff59e83343b88c1fd3a204f55a3b 0              0        1                       0   
                                                         2                       0   
                                                         3                       0   
                                                         4                       0   
                                                         5                       0   

                                                                     wi pos_i  \
id                               index_sentenca index_wi index_wj               
4918ff59e83343b88c1fd3a204f55a3b 0              0        1         Como   ADV   
                                                         2         Como   ADV   
                                                         3         Como   ADV   
                                                         4         Como   ADV   
                                                         5         Como   ADV   

                                                                         wj  \
id                               index_sentenca index_wi index_wj             
4918ff59e83343b88c1fd3a204f55a3b 0              0        1             pode   
                                                         2              ser   
                                                         3         definido   
                                                         4               um   
                                                         5          sistema   

                                                                  pos_j  \
id                               index_sentenca index_wi index_wj         
4918ff59e83343b88c1fd3a204f55a3b 0              0        1         VERB   
                                                         2          AUX   
                                                         3         VERB   
                                                         4          DET   
                                                         5         NOUN   

                                                                   cos_mean  \
id                               index_sentenca index_wi index_wj             
4918ff59e83343b88c1fd3a204f55a3b 0              0        1         0.760737   
                                                         2         0.726171   
                                                         3         0.667557   
                                                         4         0.695474   
                                                         5         0.603486   

                                                                    euc_mean  \
id                               index_sentenca index_wi index_wj              
4918ff59e83343b88c1fd3a204f55a3b 0              0        1         26.535879   
                                                         2         28.520641   
                                                         3         32.280819   
                                                         4         29.100225   
                                                         5         33.430214   

                                                                    man_mean  \
id                               index_sentenca index_wi index_wj              
4918ff59e83343b88c1fd3a204f55a3b 0              0        1         1280.3832   
                                                         2         1403.3943   
                                                         3         1582.5754   
                                                         4         1442.6814   
                                                         5         1588.4958   

                                                                    cos_max  \
id                               index_sentenca index_wi inde

## 5.3 Carregamento do arquivo de dado comparação palavra e contexto

### 5.3.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_COMPARACAO_CONTEXTO = "comparacao_contexto_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO = "comparacao_contexto_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.3.2 Cria o diretório local para receber os dados


In [ ]:
# Import das bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}".format(dirbase))
else:    
    logging.info("Diretório já existe: {}".format(dirbase))

2022-07-12 12:10:12,224 : INFO : Diretório já existe: /content/FAQUAD


### 5.3.3 Copia e descompacta os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  # Copia o arquivo de comparações do google drive para a diretório local
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO"
  
  logging.info("Terminei a cópia!")

2022-07-12 12:10:13,851 : INFO : Terminei a cópia!


Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Descompactação realizada!")

2022-07-12 12:10:19,258 : INFO : Descompactação realizada!


### 5.3.4 Carregamento das lista com os dados do arquivo


In [ ]:
#Biblioteca
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_comparacao_contexto = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_COMPARACAO_CONTEXTO, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_comparacao_contexto))) 

2022-07-12 12:10:27,335 : INFO : TERMINADO ORIGINAIS: 983942.


In [ ]:
lista_comparacao_contexto.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,cos_ctxall_mean,euc_ctxall_mean,man_ctxall_mean,cos_ctxclean_mean,...,man_ctxclean_max,cos_ctxverb_max,euc_ctxverb_max,man_ctxverb_max,cos_ctxnoun_max,euc_ctxnoun_max,man_ctxnoun_max,cos_ctxverbnoun_max,euc_ctxverbnoun_max,man_ctxverbnoun_max
274749,7bbe628a80c84b4fb95576a9dc9b181f_pert_85,26649,0,3,UFSC,PROPN,0.884805,16.329201,794.4082,0.891172,...,1060.4363,0.609301,36.278809,1708.843140,NaN,NaN,NaN,0.609301,36.278809,1708.843140
892644,df71d214297d49dfbd3acd6eb6c997e6_pert_36,82251,0,3,as,DET,0.883593,18.267374,924.5232,0.821273,...,1114.4314,0.834057,22.296152,1108.741211,0.770496,25.488766,1255.323486,0.851724,20.468782,1020.101013
862514,95a4f3a33f0748bc977f461a399d118c_pert_87,78868,0,3,divide,VERB,0.779760,29.076048,1346.8071,0.823442,...,1240.7090,1.000000,0.000000,0.000000,0.620271,36.078465,1663.575684,0.864246,24.052309,1109.050537
143813,34567128b8be4639ad4b6c43d8a26330_pert_98,13128,0,5,do,ADP,0.849598,20.479729,1019.9783,0.779343,...,1206.7844,0.673583,32.468063,1559.403076,0.763975,25.412743,1222.489380,0.778771,24.730019,1213.643066
910853,1c25ffd9584c40b986d20e796359394a_pert_63,84096,0,1,as,DET,0.852328,20.238697,1003.3666,0.802182,...,1153.7274,0.629548,41.045010,1875.444092,0.753181,26.574945,1295.293213,0.761532,27.126667,1299.442139


Remove as pontuações

In [ ]:
#lista_comparacao_contexto = lista_comparacao_contexto.query('posi != "PUNCT"')

In [ ]:
lista_comparacao_contexto.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,cos_ctxall_mean,euc_ctxall_mean,man_ctxall_mean,cos_ctxclean_mean,...,man_ctxclean_max,cos_ctxverb_max,euc_ctxverb_max,man_ctxverb_max,cos_ctxnoun_max,euc_ctxnoun_max,man_ctxnoun_max,cos_ctxverbnoun_max,euc_ctxverbnoun_max,man_ctxverbnoun_max
529755,ba270025fa8a451f9096e27ce277f9b3_pert_7,50205,0,7,em,ADP,0.815328,23.206799,1165.43430,0.800849,...,1202.4319,0.662599,31.616159,1556.729004,0.810659,23.411310,1159.006470,0.801054,23.926577,1189.031372
477319,f1146e1e59e648a8b095e7d6d7e39f7f_pert_83,44726,0,12,?,PUNCT,0.849692,19.694395,980.83185,0.828293,...,1029.1472,0.672562,31.284739,1495.914307,0.689233,28.151949,1382.194214,0.740580,25.692699,1259.815430
117392,b9d4a99cc94e4df9b52a33db272f6360_pert_83,10689,0,16,?,PUNCT,0.789268,22.956448,1141.39820,0.755945,...,1199.5818,0.681730,31.324755,1515.977051,0.605174,31.894226,1566.308105,0.701636,27.891853,1378.533936
324382,abf0abdaf96c4abca6c004133b64c916_pert_3,31516,0,7,exercícios,NOUN,0.756062,26.785437,1262.23800,0.798254,...,1170.0623,0.672650,31.373552,1481.685913,0.783288,25.440475,1215.493652,0.781854,25.511436,1208.842407
458344,a8b0acbb6f7c46b0a1346613ec913a3c_pert_57,43185,0,4,de,ADP,0.860316,18.921959,912.36755,0.789405,...,1106.6158,0.595722,39.412693,1765.531616,0.783216,23.566547,1135.663696,0.768529,24.716007,1164.075439


In [ ]:
lista_comparacao_contexto.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,cos_ctxall_mean,euc_ctxall_mean,man_ctxall_mean,cos_ctxclean_mean,...,man_ctxclean_max,cos_ctxverb_max,euc_ctxverb_max,man_ctxverb_max,cos_ctxnoun_max,euc_ctxnoun_max,man_ctxnoun_max,cos_ctxverbnoun_max,euc_ctxverbnoun_max,man_ctxverbnoun_max
247642,5f794761e2c64e6fbacde8e4c3236363_pert_16,24257,0,5,separa,VERB,0.772314,35.356743,1481.6921,0.755090,...,1509.1724,1.000000,0.000000,0.000000,0.623291,42.102203,1780.217773,0.798561,33.681763,1424.174194
389840,3f79276bb7264eb3add9616a046861c3_pert_46,36912,0,1,o,DET,0.865999,18.679306,902.0379,0.765874,...,1199.2395,NaN,NaN,NaN,0.770270,25.949106,1211.050171,0.770270,25.949106,1211.050171
33335,d880448a5a844d08a3a698c27558266a,2929,0,5,coisas,NOUN,0.791705,22.960428,1092.5442,0.790094,...,1101.9319,0.710939,27.717505,1326.556396,0.805602,22.339409,1065.708862,0.813256,21.858740,1038.755371
83250,d8de5ff4f3084fc39cdf016c4285f269_pert_85,7863,0,4,qual,PRON,0.825732,23.096899,1122.4155,0.752979,...,1315.3414,0.603223,42.970737,2053.070068,0.731704,29.042171,1388.910522,0.736555,29.015503,1383.937256
109689,1100cf4cf2b54d88bea2eac0e93df77e_pert_16,10117,0,1,que,PRON,0.786055,24.139296,1207.4648,0.690854,...,1423.2142,0.736268,28.593613,1348.959106,0.550917,34.491695,1711.714111,0.707452,27.919264,1388.389160


### 5.3.5 Criando dados indexados

In [ ]:
# Faz uma cópia da lista não indexada
lista_comparacao_contexto_indexado = lista_comparacao_contexto.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_comparacao_contexto_indexado.set_index(["id", "index_sentenca", "index_wi"], inplace=True)
lista_comparacao_contexto_indexado.head()

index_documento  \
id                               index_sentenca index_wi                    
4918ff59e83343b88c1fd3a204f55a3b 0              0                       0   
                                                1                       0   
                                                2                       0   
                                                3                       0   
                                                4                       0   

                                                                wi pos_i  \
id                               index_sentenca index_wi                   
4918ff59e83343b88c1fd3a204f55a3b 0              0             Como   ADV   
                                                1             pode  VERB   
                                                2              ser   AUX   
                                                3         definido  VERB   
                                                4               um   DET   

                                                          cos_ctxall_mean  \
id                               index_sentenca index_wi                    
4918ff59e83343b88c1fd3a204f55a3b 0              0                0.853577   
                                                1                0.843366   
                                                2                0.842175   
                                                3                0.822999   
                                                4                0.890127   

                                                          euc_ctxall_mean  \
id                               index_sentenca index_wi                    
4918ff59e83343b88c1fd3a204f55a3b 0              0               19.730730   
                                                1               20.869614   
                                                2               21.132103   
                                                3               23.358629   
                                                4               16.717981   

                                                          man_ctxall_mean  \
id                               index_sentenca index_wi                    
4918ff59e83343b88c1fd3a204f55a3b 0              0               958.74950   
                                                1               984.97650   
                                                2              1042.47640   
                                                3              1126.17190   
                                                4               830.50824   

                                                          cos_ctxclean_mean  \
id                               index_sentenca index_wi                      
4918ff59e83343b88c1fd3a204f55a3b 0              0                  0.793780   
                                                1                  0.762653   
                                                2                  0.758202   
                                                3                  0.872648   
                                                4                  0.800822   

                                                          euc_ctxclean_mean  \
id                               index_sentenca index_wi                      
4918ff59e83343b88c1fd3a204f55a3b 0              0                 23.196814   
                                                1                 25.310417   
                                                2                 25.717234   
                                                3                 20.237915   
                                                4                 22.226166   

                                                          man_ctxclean_mean  \
id                               index_sentenca index_wi                      
4918ff59e83343b88c1fd3a204f55a3b 0              0                 1122.6018   
                    

## 5.4 Medição



### 5.4.1 Wandb

https://wandb.ai/osmar-braz/MedidaCoerenciaCohebert_v1/table?workspace=user-osmar-braz

#### Função de inicialização wandb

In [ ]:
def inicializacaoWandb():

  if model_args.use_wandb:

    # Importando a biblioteca.
    import wandb

    #Login via linha de comando
    !wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project=NOME_BASE_SAIDA, name=NOME_BASE_SAIDA)

    # Atualiza os parâmetros do modelo no wandb.
    wandb.config.update(model_args)

    # Registra os parämetros não literais do model_args.
    wandb.log({'max_seq_len': model_args.max_seq_len})
    wandb.log({'do_lower_case': model_args.do_lower_case})
    wandb.log({'output_hidden_states': model_args.output_hidden_states})    
    wandb.log({"documentos_perturbados": model_args.documentos_perturbados})    
    wandb.log({"top_k_predicao": model_args.top_k_predicao}) 
    
    return wandb

### 5.4.2 Função quer realiza a medição dos documentos



#### getMedidasComparacaoPalavra

In [ ]:
import sys

def getMedidasComparacaoPalavra(id_documento, 
                                index_sentenca,
                                index_wi,
                                index_wj,
                                estrategia_medida=0):

  # Pesquisa a medida de comparação das palavras wi e wj
  # pelo id do documento, índice da sentença, índice da palavra wi e índice da palavra wj
  medidas = lista_comparacao_palavra_indexado.loc[str(id_documento),
                                                  index_sentenca,
                                                  index_wi,
                                                  index_wj]

  if len(medidas) != 0:
    # print(">>>>medidas:",medidas)
    # Seleciona a estratégia de pooling
    pooling = "_mean"
    if estrategia_medida == 1:
      pooling = "_max"

    cos = medidas["cos" + pooling]
    euc = medidas["euc" + pooling]
    man = medidas["man" + pooling]

    return cos, euc, man

  else:
    print("Problemas comparação palavras:", medidas)
    return 0, float(sys.maxsize), float(sys.maxsize)

#### Palavras Adjacentes

##### getMedidasCoerenciaPalavrasAdjacentesDocumentoTodasPalavras

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentesDocumentoTodasPalavras(id_documento,
                                                                documento, 
                                                                lista_sentenca_documento, 
                                                                lista_tokens_documento,
                                                                lista_pos_documento,
                                                                estrategia_medida):
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))

    # Quantidade de palavras da sentença
    k = len(sentenca_token)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    #Percorre as palavras do documento    
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = sentenca_token[ix]
      wj = sentenca_token[ix+1]
      # Seleciona as POS-Tagging das palavras
      pos_i = sentenca_pos[ix]
      pos_j = sentenca_pos[ix+1]

      # print("\nwi:", wi, ix)
      # print("wj:", wj, ix+1)

      # Recupera as medidas entre wi e wj             
      cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
  
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)

    # Calcula a média das medidas para as k-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasAdjacentesDocumentoSemStopWord

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentesDocumentoSemStopWord(id_documento,
                                                              documento, 
                                                              lista_sentenca_documento, 
                                                              lista_tokens_documento, 
                                                              lista_pos_documento,
                                                              estrategia_medida):
  
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    #print("sentenca:",sentenca)  
    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    #print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    #print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))
   
    # Seleciona as palavras e POS-Tagging sem as Stopwords
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, palavra in enumerate(sentenca_token):
        # Se o token estiver não lista de stopwords
        if palavra.lower() not in getStopwords(nlp):
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])              
   
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
      
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    #Percorre as palavras do documento    
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]
      wj = lista_tokens_nova[ix+1]
      # Seleciona as POS-Tagging das palavras
      pos_i = lista_pos_nova[ix]
      pos_j = lista_pos_nova[ix+1]

      # print("\nwi:", wi, ix)
      # print("wj:", wj, ix+1)                    

      # Recupera as medidas entre wi e wj             
      cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
     
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)
   
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman
    
    del lista_tokens_nova
    del lista_pos_nova    
    
  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasAdjacentesDocumentoPalavrasSalientes

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentesDocumentoPalavrasSalientes(id_documento,
                                                                    documento, 
                                                                    lista_sentenca_documento, 
                                                                    lista_tokens_documento, 
                                                                    lista_pos_documento,
                                                                    estrategia_medida,
                                                                    classe_saliente=["NOUN","VERB","AUX"]):
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    # print("Quantidade de palavras:",len(sentenca_token))

    # Seleciona somente palavras saliente 
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, pos in enumerate(sentenca_pos):
        # Se a postagging da palavra estiver na lista das classes das salientes
        if pos in classe_saliente:
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])    
       
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    #Percorre as palavras do documento    
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]
      wj = lista_tokens_nova[ix+1]
      # Seleciona as POS-Tagging das palavras
      pos_i = lista_pos_nova[ix]
      pos_j = lista_pos_nova[ix+1]

      # print("\nwi:", wi, ix)
      # print("wj:", wj, ix+1)                  

      # Recupera as medidas entre wi e wj             
      cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
     
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)

    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
    
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasAdjacentes

Recupera as medidas da coerência das palavras adjacentes de acordo com o filtro de palavras a ser utilizado nos documentos.

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentes(id_documento,
                                          documento, 
                                          lista_sentenca_documento, 
                                          lista_tokens_documento, 
                                          lista_pos_documento, 
                                          estrategia_medida = 0,
                                          filtro_palavra = 0,
                                          classe_saliente=["NOUN","VERB","AUX"]):

  """
    Recupera as medidas da coerência das palavras adjacentes de acordo com o filtro de palavras a ser utilizado nos documentos.
  """    
  
  if filtro_palavra == 0: # Todas as palavras
    return getMedidasCoerenciaPalavrasAdjacentesDocumentoTodasPalavras(id_documento, 
                                                                       documento, 
                                                                       lista_sentenca_documento, 
                                                                       lista_tokens_documento, 
                                                                       lista_pos_documento,
                                                                       estrategia_medida)
  else:
    if filtro_palavra == 1: # Sem stopwords
        return getMedidasCoerenciaPalavrasAdjacentesDocumentoSemStopWord(id_documento, 
                                                                         documento, 
                                                                         lista_sentenca_documento, 
                                                                         lista_tokens_documento, 
                                                                         lista_pos_documento,
                                                                         estrategia_medida)
    else: 
        if filtro_palavra == 2: # Somente verbos(e auxiliares) substantivos          
          return getMedidasCoerenciaPalavrasAdjacentesDocumentoPalavrasSalientes(id_documento, 
                                                                                 documento, 
                                                                                 lista_sentenca_documento, 
                                                                                 lista_tokens_documento, 
                                                                                 lista_pos_documento,
                                                                                 estrategia_medida,
                                                                                 classe_saliente=classe_saliente) 

#### Palavras Combinação

##### getMedidasCoerenciaPalavrasCombinacaoDocumentoTodasPalavras

In [ ]:
def getMedidasCoerenciaPalavrasCombinacaoDocumentoTodasPalavras(id_documento,
                                                                documento, 
                                                                lista_sentenca_documento, 
                                                                lista_tokens_documento, 
                                                                lista_pos_documento,
                                                                estrategia_medida):
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))

    # Quantidade de palavras da sentença
    k = len(sentenca_token)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0

    contaComparacoes = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = sentenca_token[ix]      
      pos_i = sentenca_pos[ix]
        
      # Percorre as palavras da sentença
      for jx in range(ix+1,k):

        # Seleciona as palavras do documento  
        wj = sentenca_token[jx]
        pos_j = sentenca_pos[jx]

        # print("\nwi:", wi, pos_i)                          
        # print("wj:", wj, pos_j)                    

        # Recupera as medidas entre wi e wj             
        cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, jx, estrategia_medida)
  
        # Acumula as medidas do par de palavras
        soma_Wcos = soma_Wcos + abs(cos)
        soma_Weuc = soma_Weuc + abs(euc)
        soma_Wman = soma_Wman + abs(man)

        contaComparacoes = contaComparacoes + 1
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(contaComparacoes)
      MWeuc = float(soma_Weuc)/float(contaComparacoes)
      MWman = float(soma_Wman)/float(contaComparacoes)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasCombinacaoDocumentoSemStopWord


In [ ]:
def getMedidasCoerenciaPalavrasCombinacaoDocumentoSemStopWord(id_documento,
                                                              documento, 
                                                              lista_sentenca_documento, 
                                                              lista_tokens_documento, 
                                                              lista_pos_documento,
                                                              estrategia_medida):
  
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    #print("sentenca:",sentenca)  
    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    #print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    #print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))
   
    # Seleciona as palavras e POS-Tagging sem as Stopwords
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, palavra in enumerate(sentenca_token):
        # Se o token estiver não lista de stopwords
        if palavra.lower() not in getStopwords(nlp):
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])              

    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k-1):
      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # Percorre as palavras da sentença
      for jx in range(ix+1,k):
        # Seleciona as palavras do documento  
        wj = lista_tokens_nova[jx]
        pos_j = lista_pos_nova[jx]

        # print("\nwi:", wi, pos_i)                          
        # print("wj:", wj, pos_j)                    

        # Recupera as medidas entre wi e wj             
        cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, jx, estrategia_medida)
  
        # Acumula as medidas do par de palavras
        soma_Wcos = soma_Wcos + abs(cos)
        soma_Weuc = soma_Weuc + abs(euc)
        soma_Wman = soma_Wman + abs(man)
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova    

  # Calcula a média das medidas para o documento
  Scos = float(soma_Scos)/float(n)
  Seuc = float(soma_Seuc)/float(n)
  Sman = float(soma_Sman)/float(n)
    
  return Scos, Seuc, Sman

##### getMedidasCoerenciaPalavrasCombinacaoDocumentoPalavrasSalientes

In [ ]:
def getMedidasCoerenciaPalavrasCombinacaoDocumentoPalavrasSalientes(id_documento,
                                                                    documento, 
                                                                    lista_sentenca_documento, 
                                                                    lista_tokens_documento, 
                                                                    lista_pos_documento,
                                                                    estrategia_medida,
                                                                    classe_saliente=["NOUN","VERB","AUX"]):
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    # print("Quantidade de palavras:",len(sentenca_token))

    # Somente palavras saliente 
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, pos in enumerate(sentenca_pos):
        # Se a postagging da palavra estiver na lista das classes das salientes
        if pos in classe_saliente:
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])
    
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
     # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k-1):
      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # Percorre as palavras da sentença
      for jx in range(ix+1,k):
        # Seleciona as palavras do documento  
        wj = lista_tokens_nova[jx]
        pos_j = lista_pos_nova[jx]

        # print("\nwi:", wi, pos_i)                          
        # print("wj:", wj, pos_j)                    

        # Recupera as medidas entre wi e wj             
        cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, jx, estrategia_medida)
  
        # Acumula as medidas do par de palavras
        soma_Wcos = soma_Wcos + abs(cos)
        soma_Weuc = soma_Weuc + abs(euc)
        soma_Wman = soma_Wman + abs(man)

    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasCombinacao

Recupera as medidas da coerência da combinação de palavras de acordo com o filtro de palavras a ser utilizado nos documentos.

In [ ]:
def getMedidasCoerenciaPalavrasCombinacao(id_documento,
                                          documento, 
                                          lista_sentenca_documento, 
                                          lista_tokens_documento, 
                                          lista_pos_documento,
                                          estrategia_medida,                                
                                          filtro_palavra = 0,
                                          classe_saliente=["NOUN","VERB","AUX"]):
  """
    Recupera as medidas da coerência da combinação de palavras de acordo com o filtro de palavras a ser utilizado nos documentos.
  """  
  
  if filtro_palavra == 0: # Todas as palavras
    return getMedidasCoerenciaPalavrasCombinacaoDocumentoTodasPalavras(id_documento, 
                                                                       documento, 
                                                                       lista_sentenca_documento, 
                                                                       lista_tokens_documento, 
                                                                       lista_pos_documento,
                                                                       estrategia_medida)
  else:
    if filtro_palavra == 1: # Sem stopwords
        return getMedidasCoerenciaPalavrasCombinacaoDocumentoSemStopWord(id_documento, documento, 
                                                                         lista_sentenca_documento, 
                                                                         lista_tokens_documento, 
                                                                         lista_pos_documento,
                                                                         estrategia_medida)
    else: 
        if filtro_palavra == 2: # Somente verbos(e auxiliares) substantivos          
          return getMedidasCoerenciaPalavrasCombinacaoDocumentoPalavrasSalientes(id_documento, 
                                                                                 documento, 
                                                                                 lista_sentenca_documento, 
                                                                                 lista_tokens_documento, 
                                                                                 lista_pos_documento,
                                                                                 estrategia_medida,
                                                                                 classe_saliente=classe_saliente)

#### Contexto

##### getMedidasComparacaoPalavrasGlobal

In [ ]:
import sys

def getMedidasComparacaoPalavrasGlobal(id_documento, 
                                       index_sentenca, 
                                       index_wi,
                                       estrategia_medida=0,
                                       filtro_palavra=0):
  
  
  # Pesquisa a medida palavra wi e o contexto
  # pelo id do documento, índice da sentença, índice da palavra wi
  medidas = lista_comparacao_contexto_indexado.loc[str(id_documento),
                                                   index_sentenca,
                                                   index_wi]

  
  if len(medidas) != 0:
    # print(">>>>medidas:",len(medidas))    
    # Seleciona a estratégia de pooling
    pooling = "_mean"
    if estrategia_medida == 1:
      pooling = "_max"

    # Seleciona o filtro de palavra
    if filtro_palavra == 0: # Todas as palavras
      cos = medidas["cos_ctxall" + pooling]
      euc = medidas["euc_ctxall" + pooling]
      man = medidas["man_ctxall" + pooling]      
    else:
      if filtro_palavra == 1: # Sem as stopwords
        cos = medidas["cos_ctxclean" + pooling]
        euc = medidas["euc_ctxclean" + pooling]
        man = medidas["man_ctxclean" + pooling]
      else:
        if filtro_palavra == 2: # Somente verbos, auxiliar e substantivo
          cos = medidas["cos_ctxverbnoun" + pooling]
          euc = medidas["euc_ctxverbnoun" + pooling]
          man = medidas["man_ctxverbnoun" + pooling]

    return cos, euc, man

  else:
    print("Problemas comparação contexto:", medidas)
    return 0, float(sys.maxsize), float(sys.maxsize)

##### getMedidasCoerenciaPalavrasGlobalDocumentoTodasPalavras

In [ ]:
def getMedidasCoerenciaPalavrasGlobalDocumentoTodasPalavras(id_documento,
                                                            documento, 
                                                            lista_sentenca_documento, 
                                                            lista_tokens_documento, 
                                                            lista_pos_documento,
                                                            estrategia_medida,
                                                            filtro_palavra):
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))

    # Quantidade de palavras da sentença
    k = len(sentenca_token)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k):

      # Seleciona as palavras do documento  
      wi = sentenca_token[ix]      
      pos_i = sentenca_pos[ix]
        
      # print("\nwi:", wi, pos_i)      

      # Recupera as medidas entre wi com o contexto
      cos, euc, man = getMedidasComparacaoPalavrasGlobal(id_documento, 
                                                         i,
                                                         ix,
                                                         estrategia_medida,
                                                         filtro_palavra)
  
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k) != 0:
      MWcos = float(soma_Wcos)/float(k)
      MWeuc = float(soma_Weuc)/float(k)
      MWman = float(soma_Wman)/float(k)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasGlobalDocumentoSemStopWord

In [ ]:
def getMedidasCoerenciaPalavrasGlobalDocumentoSemStopWord(id_documento,
                                                          documento, 
                                                          lista_sentenca_documento, 
                                                          lista_tokens_documento, 
                                                          lista_pos_documento,
                                                          estrategia_medida,
                                                          filtro_palavra):
  
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    #print("sentenca:",sentenca)  
    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    #print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    #print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))
   
    # Seleciona as palavras e POS-Tagging sem as Stopwords
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, palavra in enumerate(sentenca_token):
        # Se o token estiver não lista de stopwords
        if palavra.lower() not in getStopwords(nlp):
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])              

    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # print("\nwi:", wi, pos_i)      

      # Recupera as medidas entre wi com o contexto
      cos, euc, man = getMedidasComparacaoPalavrasGlobal(id_documento, 
                                                         i,
                                                         ix,
                                                         estrategia_medida,
                                                         filtro_palavra)
      
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k) != 0:
      MWcos = float(soma_Wcos)/float(k)
      MWeuc = float(soma_Weuc)/float(k)
      MWman = float(soma_Wman)/float(k)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  Scos = float(soma_Scos)/float(n)
  Seuc = float(soma_Seuc)/float(n)
  Sman = float(soma_Sman)/float(n)
    
  return Scos, Seuc, Sman

##### getMedidasCoerenciaPalavrasGlobalDocumentoPalavrasSalientes

In [ ]:
def getMedidasCoerenciaPalavrasGlobalDocumentoPalavrasSalientes(id_documento,
                                                                documento, 
                                                                lista_sentenca_documento, 
                                                                lista_tokens_documento, 
                                                                lista_pos_documento,
                                                                estrategia_medida,
                                                                filtro_palavra,
                                                                classe_saliente=["NOUN","VERB","AUX"]):

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    # print("Quantidade de palavras:",len(sentenca_token))

    # Somente palavras saliente 
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, pos in enumerate(sentenca_pos):
        # Se a postagging da palavra estiver na lista das classes das salientes
        if pos in classe_saliente:
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])
    
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
     # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # print("\nwi:", wi, pos_i)      

      # Recupera as medidas entre wi com o contexto
      cos, euc, man = getMedidasComparacaoPalavrasGlobal(id_documento, 
                                                         i,
                                                         ix,
                                                         estrategia_medida,
                                                         filtro_palavra)
  
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)

    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k) != 0:
      MWcos = float(soma_Wcos)/float(k)
      MWeuc = float(soma_Weuc)/float(k)
      MWman = float(soma_Wman)/float(k)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasGlobal

Recupera as medidas da palavra com o contexto global de acordo com o filtro de palavras a ser utilizado nos documentos.



In [ ]:
def getMedidasCoerenciaPalavrasGlobal(id_documento, 
                                      documento, 
                                      lista_sentenca_documento, 
                                      lista_tokens_documento, 
                                      lista_pos_documento,
                                      estrategia_medida,                             
                                      filtro_palavra = 0,
                                      classe_saliente=["NOUN","VERB","AUX"]):
  
  """
    Recupera as medidas da palavra com o contexto global de acordo com o filtro de palavras a ser utilizado nos documentos.
  """
  
  if filtro_palavra == 0: # Todas as palavras
    return getMedidasCoerenciaPalavrasGlobalDocumentoTodasPalavras(id_documento, 
                                                                   documento, 
                                                                   lista_sentenca_documento, 
                                                                   lista_tokens_documento, 
                                                                   lista_pos_documento,
                                                                   estrategia_medida,
                                                                   filtro_palavra)
  else:
    if filtro_palavra == 1: # Sem stopwords
        return getMedidasCoerenciaPalavrasGlobalDocumentoSemStopWord(id_documento, documento, 
                                                                     lista_sentenca_documento, 
                                                                     lista_tokens_documento, 
                                                                     lista_pos_documento,
                                                                     estrategia_medida,
                                                                     filtro_palavra)
    else: 
        if filtro_palavra == 2: # Somente verbos(e auxiliares) substantivos          
          return getMedidasCoerenciaPalavrasGlobalDocumentoPalavrasSalientes(id_documento, 
                                                                             documento, 
                                                                             lista_sentenca_documento, 
                                                                             lista_tokens_documento, 
                                                                             lista_pos_documento,
                                                                             estrategia_medida,
                                                                             filtro_palavra,
                                                                             classe_saliente=classe_saliente)

#### getMedidasCoerenciaDocumento

Recupera as medidas de coerência de acordo com a equação a ser utilizado nos documentos.

In [ ]:
def getMedidasCoerenciaDocumento(id_documento,                                  
                                 documento, 
                                 lista_sentenca_documento, 
                                 lista_tokens_documento, 
                                 lista_pos_documento,
                                 equacao_medida = 0,
                                 estrategia_medida = 0,
                                 filtro_palavra = 0,
                                 classe_saliente=["NOUN","VERB","AUX"]):
  
  """
    Recupera as medidas de coerência de acordo com a equação a ser utilizado nos documentos.
  """
  # print("getMedidasCoerenciaDocumento: equacao_medida", equacao_medida)
  if equacao_medida == 0: # Compara palavras adjacentes
    # print("Calculando para palavra adjacentes")
    return getMedidasCoerenciaPalavrasAdjacentes(id_documento, 
                                                 documento, 
                                                 lista_sentenca_documento, 
                                                 lista_tokens_documento, 
                                                 lista_pos_documento, 
                                                 estrategia_medida,
                                                 filtro_palavra,
                                                 classe_saliente=classe_saliente)
  else:
    if equacao_medida == 1: # Compara todas as palavras
        # print("Calculando para todas as palavras")
        return getMedidasCoerenciaPalavrasCombinacao(id_documento, 
                                                     documento, 
                                                     lista_sentenca_documento, 
                                                     lista_tokens_documento, 
                                                     lista_pos_documento, 
                                                     estrategia_medida,
                                                     filtro_palavra,
                                                     classe_saliente=classe_saliente)
    else:         
        if equacao_medida == 2: # Somente verbos(e auxiliares) substantivos          
          # print("Calculando para o contexto")  
          return getMedidasCoerenciaPalavrasGlobal(id_documento, 
                                                   documento, 
                                                   lista_sentenca_documento, 
                                                   lista_tokens_documento, 
                                                   lista_pos_documento,
                                                   estrategia_medida,
                                                   filtro_palavra, 
                                                   classe_saliente=classe_saliente)
  

### 5.4.3 Função que realiza a medição de todos os documentos

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook

def calculaMedidasDocumentos(df_lista_documentos_agrupados,                              
                             wandb, 
                             equacao_medida, 
                             estrategia_medida,
                             filtro_palavra):
    
  '''
    Percorre os documentos para calcular as medidas das sentenças
  '''

  logging.info("Processando {} pares de documentos originais e perturbados.".format(len(df_lista_documentos_agrupados)))

  # Contadores de ocorrência de coerência
  conta_Ccos = 0
  conta_Ceuc = 0
  conta_Cman = 0
  conta = 0

  # Lista para o salvamento das medidas
  lista_medidas_documentos_salvar = []

  # Barra de progresso dos documentos
  lista_documentos_agrupados_bar = tqdm_notebook(df_lista_documentos_agrupados.iterrows(), desc=f"Documentos", unit=f" documento", total=len(df_lista_documentos_agrupados))

  # Percorre os documentos do conjunto de dados
  for i, linha_documento in lista_documentos_agrupados_bar:  
    # if i < 10:     

      # Conta o número de pares de documentos
      conta = conta + 1

      #print("linha_documento:",linha_documento)
      # Recupera o id do documento Original
      id_documento_original = linha_documento[0]   

      # print("id_documento_original:",id_documento_original)    
      lista_sentenca_documento_original = linha_documento[1]     
      #print("lista_sentenca_documento_original:",lista_sentenca_documento_original)
      #print("len(lista_sentenca_documento_original):",len(lista_sentenca_documento_original))
      # Recupera o documento Original
      documento_original = linha_documento[2]    
      #print("documento_original:",documento_original) 
      # Recupera os tokens do documento original
      lista_tokens_documento_original = linha_documento[3]
      #print("lista_tokens_documento_original:",lista_tokens_documento_original)
      #print("len(lista_tokens_documento_original):",len(lista_tokens_documento_original))
      # Recupera o postagging do documento original
      lista_pos_documento_original = linha_documento[4]
      #print("lista_pos_documento_original:",lista_pos_documento_original)
      #print("len(lista_pos_documento_original):",len(lista_pos_documento_original))

      Ccos_orig, Ceuc_orig, Cman_orig = getMedidasCoerenciaDocumento(id_documento_original,
                                                                     documento_original, 
                                                                     lista_sentenca_documento_original,
                                                                     lista_tokens_documento_original,
                                                                     lista_pos_documento_original,
                                                                     equacao_medida=equacao_medida,
                                                                     estrategia_medida=estrategia_medida,
                                                                     filtro_palavra=filtro_palavra)
               
      # Recupera o id do documento Perturbado
      id_documento_perturbado = linha_documento[5]     
      # print("id_documento_perturbado:",id_documento_perturbado)    
      lista_sentenca_documento_perturbado = linha_documento[6]     
      #print("lista_sentenca_documento_perturbado:",lista_sentenca_documento_perturbado)
      #print("len(lista_sentenca_documento_perturbado):",len(lista_sentenca_documento_perturbado))
      # Recupera o documento Perturbado
      documento_perturbado = linha_documento[7]
      #print("documento_perturbado:",documento_perturbado) 
      # Recupera os tokens do documento perturbado
      lista_tokens_documento_perturbado = linha_documento[8]
      #print("lista_tokens_documento_perturbado:",lista_tokens_documento_perturbado)
      #print("len(lista_tokens_documento_perturbado):",len(lista_tokens_documento_perturbado))
      # Recupera o postagging do documento original
      lista_pos_documento_perturbado = linha_documento[9]
      #print("lista_pos_documento_perturbado:",lista_pos_documento_perturbado)
      #print("len(lista_pos_documento_perturbado):",len(lista_pos_documento_perturbado))
      
      Ccos_pert, Ceuc_pert, Cman_pert =  getMedidasCoerenciaDocumento(id_documento_perturbado,
                                                                      documento_perturbado, 
                                                                      lista_sentenca_documento_perturbado, 
                                                                      lista_tokens_documento_perturbado, 
                                                                      lista_pos_documento_perturbado,
                                                                      equacao_medida=equacao_medida,
                                                                      estrategia_medida=estrategia_medida,
                                                                      filtro_palavra=filtro_palavra)
    
      # Verifica a medida de coerência Ccos(similaridade do cosseno) das sentenças do documento original com as sentenças do documento perturbado.
      # Quanto maior o valor de Ccos mais as orações do documentos são coerentes
      if Ccos_orig >= Ccos_pert:
          conta_Ccos = conta_Ccos + 1

      # Verifica a medida de incoerência Seuc(distância euclidiana) das sentenças do documento original com as sentenças do documento perturbado.
      # Quanto menor o valor de Ceuc mais as orações do documentos são coerentes
      if Ceuc_orig <= Ceuc_pert:
          conta_Ceuc = conta_Ceuc + 1

      # Verifica a medida de incoerência Sman(distância de manhattan) das sentenças do documento original com as sentenças do documento perturbado.
      # Quanto menor o valor de Cman mais as orações do documentos são coerentes
      if Cman_orig <= Cman_pert:
          conta_Cman = conta_Cman + 1        

      # Guarda as medidas em uma lista para salvar em arquivo
      # Guarda as medidas dos documentos originais
      lista_medidas_documentos_salvar.append([id_documento_original, 
                                              Ccos_orig,  
                                              Ceuc_orig,  
                                              Cman_orig])
      # Guarda as medidas dos documentos perturbados
      lista_medidas_documentos_salvar.append([id_documento_perturbado, 
                                              Ccos_pert, 
                                              Ceuc_pert, 
                                              Cman_pert])
      
  logging.info("Total de Pares : {}.".format(str(conta)))

  if model_args.use_wandb:
       wandb.log({'pares_doc': conta})

  logging.info("Pares Corretos Ccos: {}.".format(str(conta_Ccos)))
  acuracia_Ccos = float(conta_Ccos)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_Ccos*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_ccos': acuracia_Ccos})

  logging.info("Pares Corretos Ceuc: {}.".format(str(conta_Ceuc)))
  acuracia_Ceuc = float(conta_Ceuc)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_Ceuc*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_ceuc': acuracia_Ceuc})  

  logging.info("Pares Corretos Cman: {}.".format(str(conta_Cman)))
  acuracia_Cman = float(conta_Cman)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_Cman*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_cman': acuracia_Cman})  

  logging.info("TERMINADO!")

  del lista_documentos_agrupados_bar
 
  return lista_medidas_documentos_salvar, conta, acuracia_Ccos, conta_Ccos, acuracia_Ceuc, conta_Ceuc, acuracia_Cman, conta_Cman

### 5.4.4 Salvando os resultados

#### Salvando o resultado da medição

In [ ]:
def salvaResultadoMedicao(lista_medidas_documentos_salvar):

  if model_args.salvar_medicao:

    # Import das bibliotecas.
    import os
    import datetime

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    EQUACAO = '_adj' # adjacentes
    if model_args.equacao_medida == 1:
        EQUACAO = '_com' # combinação de todas as palavras
    else:
      if model_args.equacao_medida == 2: 
         EQUACAO = '_ctx' # contexto

    ESTRATEGIA = '_mean'
    if model_args.estrategia_medida == 1:
      ESTRATEGIA = '_max'

    FILTRO_PALAVRA = '_tap' # Todas as palavras
    if model_args.filtro_palavra == 1:
      FILTRO_PALAVRA = '_ssw'  # Sem stopwords     
    else:
      if model_args.filtro_palavra == 2:
        FILTRO_PALAVRA = '_svs'  # Somente verbos(e auxiliares) e substantivos

    AJUSTADO = '_pretreinado'
    if model_args.usar_mcl_ajustado == True:
        AJUSTADO = '_ajustado'

    # Contatena os parâmetros que forma o nome do arquivo medição    
    NOME_ARQUIVO_MEDICAO = NOME_BASE_SAIDA + "_P_" + str(model_args.documentos_perturbados) + "_K_" + str(model_args.top_k_predicao) + EQUACAO + ESTRATEGIA + FILTRO_PALAVRA + getNomeModeloBERT(model_args) + getTamanhoBERT(model_args) + AJUSTADO 
    
    # Diretório do drive do arquivo
    DIRETORIO_MEDICAO_DRIVE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Medicao/"

    # Diretório local para salvar o arquivo
    DIRETORIO_MEDICAO_LOCAL = DIRETORIO_LOCAL + "Medicao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_MEDICAO_DRIVE):  
      # Cria o diretório
      os.makedirs(DIRETORIO_MEDICAO_DRIVE)
      logging.info("Diretório criado: {}.".format(DIRETORIO_MEDICAO_DRIVE))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_MEDICAO_DRIVE))

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_MEDICAO_LOCAL):  
      # Cria o diretório
      os.makedirs(DIRETORIO_MEDICAO_LOCAL)
      logging.info("Diretório criado: {}.".format(DIRETORIO_MEDICAO_LOCAL))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_MEDICAO_LOCAL))

    # Caminho completo do arquivo compactado no drive
    NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO = DIRETORIO_MEDICAO_DRIVE + NOME_ARQUIVO_MEDICAO + ".zip"
    # print("NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO:", NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO)

    # Caminho completo do arquivo compactado no local
    NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO = DIRETORIO_MEDICAO_LOCAL + NOME_ARQUIVO_MEDICAO + ".zip"
    # print("NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO:", NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO)

    # Caminho completo do arquivo no local
    NOME_ARQUIVO_MEDICAO_LOCAL = DIRETORIO_MEDICAO_LOCAL + NOME_ARQUIVO_MEDICAO + ".csv"
    # print("NOME_ARQUIVO_MEDICAO_LOCAL:", NOME_ARQUIVO_MEDICAO_LOCAL)

    # Gera todo o conteúdo a ser salvo no arquivo
    novo_conteudo = ''        
    for resultado in lista_medidas_documentos_salvar:      
      novo_conteudo = novo_conteudo + data_e_hora.strftime('%d/%m/%Y %H:%M') + ';' + str(resultado[0]) + ';'  + str(resultado[1]) + ';'  + str(resultado[2]) + ';'  + str(resultado[3]) + '\n'

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO):
      # Copia arquivo da medição compactado do google drive para o drive local
      !cp "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO"  
      # Descompacta arquivo da medição compactado no drive local
      !unzip -o -j -q "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" -d "$DIRETORIO_MEDICAO_LOCAL"

      logging.info("Atualizando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_LOCAL))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_MEDICAO_LOCAL,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_COMPLETO,'w')
      # Escreve o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()

      # Compacta o arquivo da medição
      !zip -o -q -j "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_LOCAL"
      # Copia o arquivo da medição compactado para o drive
      !cp "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO"  

    else:
      logging.info("Criando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_LOCAL))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_LOCAL,'w')
      arquivo.writelines('data;arquivo;ccos;ceuc;cman\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

      # Compacta o arquivo da medição
      !zip -o -q -j "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_LOCAL"
      # Copia o arquivo da medição compactado para o drive
      !cp "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO"

#### Salvando o resultado da avaliação

In [ ]:
def salvaResultadoAvaliacao(tempo_total_processamento, 
                            conta, 
                            acuracia_Ccos, 
                            conta_Ccos, 
                            acuracia_Ceuc, 
                            conta_Ceuc, 
                            acuracia_Cman, 
                            conta_Cman):

  if model_args.salvar_avaliacao:

    # Import das bibliotecas.
    import os

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    EQUACAO = '_adj' # adjacentes
    if model_args.equacao_medida == 1:
        EQUACAO = '_com' # combinação de todas as palavras
    else:
      if model_args.equacao_medida == 2: 
         EQUACAO = '_ctx' # contexto

    ESTRATEGIA = '_mean'
    if model_args.estrategia_medida == 1:
      ESTRATEGIA = '_max'

    FILTRO_PALAVRA = '_tap' # Todas as palavras
    if model_args.filtro_palavra == 1:
      FILTRO_PALAVRA = '_ssw'  # Sem stopwords     
    else:
      if model_args.filtro_palavra == 2:
        FILTRO_PALAVRA = '_svs' # Somente verbos(e auxiliares) e substantivos   

    AJUSTADO = '_pretreinado'
    if model_args.usar_mcl_ajustado == True:
        AJUSTADO = '_ajustado'

    # Nome arquivo resultado    
    NOME_ARQUIVO_AVALIACAO = NOME_BASE_SAIDA + "_P_" + str(model_args.documentos_perturbados) + "_K_" + str(model_args.top_k_predicao) + EQUACAO + ESTRATEGIA + FILTRO_PALAVRA + getNomeModeloBERT(model_args) + getTamanhoBERT(model_args) + AJUSTADO

    # Diretório para salvar o arquivo de resultado.
    DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_medicao_palavra/Avaliacao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_AVALIACAO):  
      # Cria o diretório
      os.makedirs(DIRETORIO_AVALIACAO)
      logging.info("Diretório criado: {}.".format(DIRETORIO_AVALIACAO))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_AVALIACAO))

    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_AVALIACAO_COMPLETO = DIRETORIO_AVALIACAO + NOME_ARQUIVO_AVALIACAO + '.csv'

    # Conteúdo a ser adicionado.
    novo_conteudo = NOME_ARQUIVO_AVALIACAO + ';' + data_e_hora.strftime('%d/%m/%Y %H:%M') + ';' + tempo_total_processamento + ';'  + str(conta) + ';'  + str(acuracia_Ccos) + ';' + str(conta_Ccos) + ';' + str(acuracia_Ceuc) + ';' + str(conta_Ceuc) + ';' + str(acuracia_Cman) + ';' + str(conta_Cman) + '\n'

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_AVALIACAO_COMPLETO):
      logging.info("Atualizando arquivo resultado avaliação: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      # escreva o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()
    else:
      logging.info("Criando arquivo resultado avaliação: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')      
      arquivo.writelines('arquivo;data;tempo;conta;ccos;contaccos;ceuc;contaceuc;cman;contacman\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

### 5.4.5 Função de cálculo das medidas dos documentos

In [ ]:
# Import das bibliotecas.
import time
import datetime
import gc

def procedimentoCalculaMedida(equacao_medida, 
                              estrategia_medida, 
                              filtro_palavra):

  # Seta o parâmetro da equação
  model_args.equacao_medida = equacao_medida
  # Seta o parâmetro da estratégia
  model_args.estrategia_medida = estrategia_medida
  # Seta o parâmetro do fitro
  model_args.filtro_palavra = filtro_palavra

  logging.info("Processamento a equação {}, com estratégia {} e com filtro palavra {}.".format(EQUACAO_MEDIDA_STR[equacao_medida], ESTRATEGIA_MEDIDA_STR[estrategia_medida], FILTRO_PALAVRAS_STR[filtro_palavra]))
      
  # Marca o tempo de início do processamento
  tempoInicioTeste = time.time()
  logging.info("Tempo início processamento: {:} (h:mm:ss).".format(formataTempo(tempoInicioTeste)))

  # Inicializa o wandb para registro
  wandb = inicializacaoWandb()
  
  # Calcula as medidas dos documentos  
  resultado_medida, conta, acuracia_Ccos, conta_Ccos, acuracia_Ceuc, conta_Ceuc, acuracia_Cman, conta_Cman = calculaMedidasDocumentos(df_lista_documentos_agrupados, wandb, equacao_medida, estrategia_medida, filtro_palavra)
  # print("resultado_medida:", resultado_medida)

  # Pega o tempo atual menos o tempo do início do processamento.
  tempoFinalTeste = time.time()
  tempo_total_processamento = formataTempo(tempoFinalTeste - tempoInicioTeste)

  # Salva o resultado da classificação
  salvaResultadoMedicao(resultado_medida)
  
  # Salva o resultado da avaliação
  salvaResultadoAvaliacao(tempo_total_processamento, conta, acuracia_Ccos, conta_Ccos, acuracia_Ceuc, conta_Ceuc, acuracia_Cman, conta_Cman)

  logging.info("  Tempo processamento: {:} (h:mm:ss).\n".format(tempo_total_processamento))

  # Finaliza o wandb
  if model_args.use_wandb:
     wandb.finish()  

  # Apaga as variáveis
  del resultado_medida  
  del wandb

  # Chama o coletor de lixo para esvaziar a memória
  gc.collect()  

### 5.4.6 Executa o procedimento para todos os parâmetros

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook

# Barra de progresso equação(0 - Adjacentes(CAW), 1 - Combinação(CCW), 2 - Globa (CGW))
equacao_medida_bar = tqdm_notebook(enumerate(EQUACAO_MEDIDA), desc=f'Equações', unit=f'equação', total=len(EQUACAO_MEDIDA))

# Percorre todos as equações a serem avaliados 
for equacao_medida_i, equacao_medida in equacao_medida_bar:

  # Barra de progresso estratégias(0 - MEAN e 1 - MAX para as palavras formadas por mais de um token do BERT)
  estrategia_medida_bar = tqdm_notebook(enumerate(ESTRATEGIA_MEDIDA), desc=f'Estratégias', unit=f'estratégia', total=len(ESTRATEGIA_MEDIDA))

  # Percorre todos as estratégias a serem avaliados
  for estrategia_medida_i, estrategia_medida in estrategia_medida_bar:

    # Barra de progresso filtro (0 - ALL, 1 - CLEAN, 2 - VERBNOUN)
    filtro_palavra_bar = tqdm_notebook(enumerate(FILTRO_PALAVRAS), desc=f'Filtro palavras', unit=f'filtro', total=len(FILTRO_PALAVRAS))

    # Percorre todos formas de filtro de palavras a serem avaliados
    for filtro_palavra_i, filtro_palavra in filtro_palavra_bar:
      
      # Passa os parâmetros para o procedimento cálculo das medidas
      procedimentoCalculaMedida(equacao_medida, estrategia_medida, filtro_palavra)

Equações:   0%|          | 0/3 [00:00<?, ?equação/s]

Estratégias:   0%|          | 0/2 [00:00<?, ?estratégia/s]

Filtro palavras:   0%|          | 0/3 [00:00<?, ?filtro/s]

2022-07-12 12:10:29,275 : INFO : Processamento a equação ADJACENTES, com estratégia MEAN e com filtro palavra TODAS_AS_PALAVRAS.
2022-07-12 12:10:29,278 : INFO : Tempo início processamento: 19185 days, 12:10:29 (h:mm:ss).


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


2022-07-12 17:33:16,720 : INFO : Processando 90000 pares de documentos originais e perturbados.


Documentos:   0%|          | 0/90000 [00:00<?, ? documento/s]

2022-07-12 17:36:59,080 : INFO : Total de Pares : 90000.
2022-07-12 17:36:59,083 : INFO : Pares Corretos Ccos: 48876.
2022-07-12 17:36:59,091 : INFO : Acurácia: 54.30666666666667.
2022-07-12 17:36:59,098 : INFO : Pares Corretos Ceuc: 61769.
2022-07-12 17:36:59,108 : INFO : Acurácia: 68.63222222222221.
2022-07-12 17:36:59,112 : INFO : Pares Corretos Cman: 58598.
2022-07-12 17:36:59,123 : INFO : Acurácia: 65.1088888888889.
2022-07-12 17:36:59,129 : INFO : TERMINADO!
2022-07-12 17:36:59,140 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/FAQUAD/validacao_medicao_palavra/Medicao/.
2022-07-12 17:43:56,566 : INFO : Criando arquivo medição: /content/drive/MyDrive/Colab Notebooks/Data/FAQUAD/validacao_medicao_palavra/Medicao/MedidaCoerenciaFaquad_v1_P_100_K_100_ctx_max_svs_BERTimbau_large_pretreinado.csv.
2022-07-12 17:43:59,353 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/FAQUAD/validacao_medicao_palavra/Avaliacao/.
2022-07-12 17:43:59,35

acuracia_ccos,▁
acuracia_ceuc,▁
acuracia_cman,▁
do_lower_case,▁
documentos_perturbados,▁
max_seq_len,▁
output_hidden_states,▁
pares_doc,▁
top_k_predicao,▁
acuracia_ccos,0.54307
acuracia_ceuc,0.68632


# 6 Finalização

## 6.1 Tempo final de processamento



In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print('')
print('  Tempo processamento:  {:} (h:mm:ss)'.format(tempo_total_processamento))


  Tempo processamento:  5:37:11 (h:mm:ss)
